This project centers on constructing an Information Retrieval (IR) system using the Block-Sorted Based Indexing (BSBI) algorithm. The BSBI algorithm facilitates the creation of an inverted index from a collection of text documents, and the objective is to implement key steps, including document preprocessing, term indexing, and the merging of intermediate index blocks. Also, Gamma codeing is implemented which is a variable-length coding technique, will optimize the representation of positive integers, improving storage efficiency and speeding up information retrieval, particularly for datasets with skewed distributions where smaller values are more common.

<div style="line-height: 2;">For developing this system, the first step is reading document files which are prepared. But before that, we need to import some neccessary libraries:
    <ul>
        <li>
            <b>NLTK</b>: Natural Language ToolKit: A leading platform for building Python programs to work with human language data
        </li>
        <li>
            <b>OS</b>: A built-in module that provides a wide range of functions for interacting with the operating system. It allows you to work with file systems, directories, paths, environment variables, and more.
        </li>
        <li>
            <b>String</b>: A package which provides a wide range of string manipulation functions and methods
        </li>
    </ul>

Also we import some specific parts of NLTK which are described later:
<ul>
    <li>
        <b>Stopwords</b>
    </li>
    <li>
        <b>Regexp_tokenize</b>
    </li>
    <li>
        <b>shutil</b>
    </li>
    <li>
        <b>PorterStemmer</b>
    </li>
    <li>
        <b>json</b>
    </li>
</ul>

In [1]:
import os
import string
import nltk

In [2]:
# Define path of data file here
folder_path = "data"

In [3]:
# make a list of all files in "data" directory which are our text files
file_list = os.listdir(folder_path)

In [4]:
# text_files stores text files' names
text_files = [file for file in file_list if file.endswith('.txt')]

# print text files titles
print("document titles:\n" + "-"*40)
for text_file in text_files:
    print(text_file)
print( "-"*40)

document titles:
----------------------------------------
Jerry Decided To Buy a Gun.txt
Rentals at the Oceanside Community.txt
Gasoline Prices Hit Record High.txt
Cloning Pets.txt
Crazy Housing Prices.txt
Man Injured at Fast Food Place.txt
A Festival of Books.txt
Food Fight Erupted in Prison.txt
Better To Be Unlucky.txt
Sara Went Shopping.txt
Freeway Chase Ends at Newsstand.txt
Trees Are a Threat.txt
A Murder-Suicide.txt
Happy and Unhappy Renters.txt
Pulling Out Nine Tons of Trash.txt
----------------------------------------


As it this part I want to divide document files 3 by 3, I imported <b>json</b> package here.

In [5]:
# divide data into 5 directories. inverted index of each block will be stored in its corresponding folder.
import shutil

cnt = 0
index = 1
blocks = []

# Divid documents 3 by 3 and add them to their corresponding folder
for text_file in text_files:

    # Make a new directory named "block%index"
    if cnt % (3) == 0:
        os.mkdir("data/block%i" % (index))
        
        # Append block number ot the list of blocks
        blocks.append(index)

    # Definde path of new directory which we want to move our data
    new_path = os.path.join(folder_path, 'block%i' % index)

    # Definde path of old file which we want to move our data in it
    old_path = os.path.join(folder_path, text_file)

    # Move documents from "data" directory to the "block%index"
    shutil.move(old_path, new_path)

    # Increment cnt
    cnt += 1

    # If 3 docs are moved to the block%index folder, increment index
    if cnt % 3 == 0: index += 1

#### Preprocessing

<div style="line-height: 2;">
In this step, we want to preprocess text files functionally. Below steps have to be implemented:
    <ol>
        <li>
            <b>tokenization</b>: which we will do it using `regexp_tokenize()` function from `NLTK` library. For this job, we need to specify a pattern for natural language. As in our documents we have words, numbers with less or equal to 3 digits, numbers with more than 3 digits which are seperated 3 by 3, and floating point numbers, we have to use a pattern to handle all these types concurrently. We used `r'\d{1,3}(?:,\d{3})*(?:\.\d+)?|\w+'` which is decoded below: <br>
            <ul>
                <li>
                    \d{1,3}: This part matches one to three-digit numbers like "4", "32", "879", not "1234" or "1,000".
                </li>
                <li>
                (?:,\d{3})*: This part matches comma-separated thousands separators for numbers like "75,000".
                </li>
                <li>
                (?:\.\d+)?: This part handles floating point numbers.
                </li>
                <li>
                |: This symbol acts as an OR operator, allowing the regular expression to match either a number (as described above) or its next part.
                </li>
                <li>
                \w+: This part matches words, which consist of one or more word characters (letters, digits, and underscores). It's not limited to numbers and can match words like "apple," "word123," and "my_variable."
                </li>
            </ul>
        </li>
      <li>
          <b>Lowercase tokens</b>: as step of preprocessing, we have to convert all words' characters to lowercase. So, after query preprocessing, our job is easier.      
      </li>
        <li>
         <b>Delete Stopwrds</b>: We have to delete stopwords(These are the words which are use frequently in documents and not neccessary in Information Retrival. There is a list of stopwords which is prepared for English and we downloaded above.). So, the process of retriving will be faster because there are less words for checking.
        </li>
        <li>
           <b>Delete punctuations</b>: Trivially, punctuations are not needed when we want to retrive information. So, it is needed to omit all punctuations from all documents.
           </li>
<li>
           <b>Stemming</b>: The process of stemming a documnet will be done by stemming() function.
           </li>
    </ol>
</div>

In [6]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords

Import <b>PorterStemmer</b> from <b>NLKT</b> to do the stemming.

In [7]:
from nltk.stem import PorterStemmer

#### Stemming

This function will do the stemming process for all words in a document and returns the new documnet.

In [8]:
def stemming(document):
    ps = PorterStemmer()
    
    for word in document.split():
        word = ps.stem(word)
    return document

In the bemow fumction, we w

In [9]:
def preprocessing(document):
    '''
    Define a regular expression pattern for tokenization 
    (matching words, 3-digit numbers, numbers with more than 3 digits and 
    have thousands seperators in which n > 3, and floating-point numbers)
    '''
    pattern = r'\d{1,3}(?:,\d{3})*(?:\.\d+)?|\w+'

    # Tokenize the text using the regular expression pattern    
    tokens = regexp_tokenize(document, pattern)

    # Lowercase the tokens
    tokens = [word.lower() for word in tokens]

    # Get the set of English stopwords
    stop_words = set(stopwords.words('english'))

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]

    # Stemming
    document = stemming(' '.join(tokens))

    # return the processed document
    return document
    

#### Inverted index

For implementing the BSBI algorithm for inverted indexing, we make inverted indices for documnets in each block. In each inveeted index, we store terms as keys of dictionary and docIDs as postings. But the point is that these docIDs are not simple integers, they are gamma encoded and are some coded binary strings.

Also, as I want to store blocks' inverted indicies in a json file as a dictionary, I imported json.

In [10]:
import json

This function will do the procedure of gamma encoding for a given integer docID.

In [11]:
def gamma_encoder(docID):
    docID_copy = docID

    # set length of the unary and binary parts as 0
    length = 0

    # While docID > 1, we will divide it by 2 to gain its binary representation.
    while docID > 1:
        docID //= 2
        length += 1
        
    # Make unery part with '0's and end them with s '1' as it is expected in gamma encoding
    unery_part = '0' * length + '1'

    # Find re reminder; actually find docID - 2^length
    binary_part_num = docID_copy - 2 ** length

    # Define an empty string to store binary part
    binary_rep  = ""

    # Make binary form of binary_part_num in a loop
    while binary_part_num > 0:
        r = binary_part_num % 2
        binary_part_num //= 2

        # If the reminder is 0, then we insert a 0 at the begining of the string
        if r == 0:
            binary_rep = "0" + binary_rep
        
        # IF the reminder is 1, then we insert a 1 at the begining of the string
        else: binary_rep = "1" + binary_rep

    # If binary part was not represented in "length" chars, insert some zeros at its begining.
    if len(binary_rep) < length:
        diff = length - len(binary_rep)
        binary_rep = "0" * diff + binary_rep
    
    # Return the coded string
    return unery_part + binary_rep



The below function will decode a gamma codded string  and return an integer.

In [12]:
def gamma_decoder(gamma_code):

    # Find power of 2 in the gamma code formula
    length = 0

    # find number of 0s at the begining of the coded string which is same as power of 2 in the gamma code formula
    while length < len(gamma_code) and gamma_code[length] == '0':
        length += 1
    
    # Split binary part of the string and convert it to its corresponding integer
    binary_part = gamma_code[length+1:]
    reminder = 0
    for digit in binary_part:
        if digit == '1':
            reminder = reminder * 2 + 1
        else: reminder = reminder * 2
    
    # Return decoded integer
    return 2 ** length + reminder

In [13]:
# Create an empty list which will contain docIDs
docIDs = []

# Define a variable for allocating docIDs to the files
id = 1

# Define number of blocks which we want to divide our files into.
docs_per_block = 3

# Make inverted index for documents in each block.
for block in blocks:

    block_inverted_index = {}

    # Go to the corresponding block
    block_path = os.path.join(folder_path, "block%i" % block)
    
    # Make a list of block's files
    block_list = os.listdir(block_path)
    block_files = [file for file in block_list if file.endswith('.txt')]

    # Open each document in each block, do the preprocessing procedure, and make its inverted index.
    for text_file in block_files:

        docIDs.append(id)

        # Fild path of documnet file to open
        file_path = os.path.join(block_path, text_file)

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin-1') as file:
                file_content = file.read()
            
        # Documnet preprocessing 
        processed_text = preprocessing(file_content)

        for term in processed_text.split():

        # For each term in the document, code its postings values
            gamma_encoded = gamma_encoder(id)

            # If term is in the file's corresponding block inverted index, check if the docID is in postings or not
            if term in block_inverted_index:
                
                # If docID is not in the postings, then append it to the postings
                if gamma_encoded not in block_inverted_index[term]:
                    block_inverted_index[term].append(gamma_encoded)
                
                # else, do nothing
                else: continue

            # If the docID is not in postings, then add it to the list
            else:
                block_inverted_index[term] = [gamma_encoded]
        id += 1

    # At the end, make a json file to store the block's iverted index in the disk
    json_path = os.path.join(block_path, "inverted_index%i.json" % block)
    with open(json_path, 'w') as json_file:
        json.dump(block_inverted_index, json_file)

## Merging inverted indices

In [14]:
# Merge 2 sorted lists which will give a sorted array of docIDs
def merge(docID1, docID2):
    decoded_docID1 = []
    decoded_docID2 = []

    # Decode all gamma codes in docID1
    for id in docID1:
        decoded_docID1.append(gamma_decoder(id))

    # Decode all gamma codes in docID2
    for id in docID2:
        decoded_docID2.append(gamma_decoder(id))

    docID1 = decoded_docID1
    docID2 = decoded_docID2

    # Start merging process
    result = []
    i = 0
    j = 0
    while i < len(docID1) and j < len(docID2):
        if docID1[i] == docID2[j]:
            result.append(docID1[i])
            i += 1
            j += 1
        elif docID1[i] < docID2[j]:
            result.append(docID1[i])
            i += 1
        else:
            result.append(docID2[j])
            j += 1
    if i == len(docID1):
        if j < len(docID2):
            for k in range(j, len(docID2)):
                result.append(docID2[k])
    if j == len(docID2):
        if i < len(docID1):
            for k in range(i, len(docID1)):
                result.append(docID1[k])
    
    # return the merged list of encoded postings
    encoded_result = []
    for i in range(len(result)):
        encoded_result.append(gamma_encoder(result[i]))
    return encoded_result

In the below block, we will merge blocks' inverted indices and store the merged dictionary on the disk. The point is that as blocks' inverted indices were created in the order of sorted docIDs, mering in this example is not neccessary. Because, lists does not have any intersection and each of them is sorted. Also, docIDs in block(i) inverted index are all smaller than docIDs in block(i+1) inverted index. But as it is desired to implement the algorithm completely, we merge them in O(n).

In fact, when I want to create the merged_inverted_index, I check if the current key in block_inverted_index is in the merged_inverted_index or not. If it is not there, then add the fisrt docID in block_inverted_index as the started docID, and for the rest of docIDs in block_inverted_index, compute their difference with the previous docID in the merged_inverted_index and then, append the difference to the merged_inverted_index. If the key is in the merged_inverted_index, I compute the docID of the last documnt which the key happened there(sum of the first docID and differences) and then, compute the difference between current docID and the last one in the merged_inverted_index. Finally, append the new difference to the merged_inverted_index.


The point is that as we store differences, merging these values is not a good idea because then, the order of docIDs will disappear. So, as docIDs in block_inverted_indcies are in an increasing order, we prefer to append the new difference to the list instead of merge lists.

But, if merging is required, there is a block of commented code in the below section which makes a merged_inverted_index dictionary in which the values for each key are as follows: the first docID which the key is appeared in, is the fisrt elemnt of the list and for other values, their difference with the first docID is stored in the list to do not change the order and merging 2 sorted lists will be possible.

In [24]:
merged_inverted_index = {}

for block in blocks:
    # Open the inverted index corresponding to each block
    inverted_index_path = os.path.join(folder_path + "/block%i" % block, "inverted_index%i.json" % block)

    with open(inverted_index_path, 'r') as json_file:
        block_inverted_index = json.load(json_file)

        # find keys of the block's inverted index dictionary
        keys = []
        for key in block_inverted_index.keys():
            keys.append(key)

        # For each key in keys, check wheather the key is in the merged list or not
        for key in keys:

            # If the key is in merged_inverted_index, just add its posting to its corresponding posting
            if key in merged_inverted_index:
                for id_ in block_inverted_index[key]:
                    sum_ = 0
                    for dID in merged_inverted_index[key]:
                        sum_ += gamma_decoder(dID)
                    difference = gamma_decoder(id_) - sum_
                    merged_inverted_index[key] = merged_inverted_index[key] + [gamma_encoder(difference)]            
            
            # If the key is not in merged_inverted_index, append it and add its posting to its corresponding posting
            else:
                fisrt_docID = block_inverted_index[key][0]
                merged_inverted_index[key] = [fisrt_docID]
                for k in range(1, len(block_inverted_index[key])):
                    difference = gamma_decoder(block_inverted_index[key][k]) - gamma_decoder(block_inverted_index[key][k-1])                   
                    merged_inverted_index[key] = merged_inverted_index[key] + [gamma_encoder(difference)]            


# If you want to store differences of docIDs with 1st one, except 1st one which is itsel in tne merged list, run below commented code:
''' In fact, in each list of values corresponding to each key of the merged_inverted_index dictionary,
the first value is the docID of the first documnet which the key happened and i-th value(i != 0) is 
the difference between i-th document which the key happened there and the docID of the 1st documnet 
which the key happened. '''
# min_value = 0
# for key in merged_inverted_index:
#     for index in range(len(merged_inverted_index[key])):
#         if index == 0:
#             min_value = merged_inverted_index[key][index]
#         else:
#             merged_inverted_index[key][index] = gamma_encoder(gamma_decoder(merged_inverted_index[key][index]) - gamma_decoder(min_value))


# store final inverted index in a seperate file on disk
merged_inverted_index_path = os.path.join(folder_path, "inverted_index.json")
with open(merged_inverted_index_path, 'w') as json_file:
    json.dump(merged_inverted_index, json_file)